<a href="https://colab.research.google.com/github/appl044/BowlerStudioConfiguration/blob/master/quick_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VideoReTalking：Audio-based Lip Synchronization for Talking Head Video Editing In the Wild

[Arxiv](https://arxiv.org/abs/2211.14758) | [Project](https://vinthony.github.io/video-retalking/) | [Github](https://github.com/vinthony/video-retalking)

Kun Cheng, Xiaodong Cun, Yong Zhang, Menghan Xia, Fei Yin, Mingrui Zhu, Xuan Wang, Jue Wang, Nannan Wang

Xidian University, Tencent AI Lab, Tsinghua University

*SIGGRAPH Asia 2022 Conferenence Track*



**Installation** (30s)

In [1]:
#@title
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi

!python --version
!apt-get update
!apt install ffmpeg &> /dev/null

print('Git clone project and install requirements...')
!git clone https://github.com/vinthony/video-retalking.git &> /dev/null
%cd video-retalking
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt

Thu Dec 12 21:24:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Download Pretrained Models**

In [2]:
#@title
print('Download pre-trained models...')
!mkdir ./checkpoints
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/30_net_gen.pth -O ./checkpoints/30_net_gen.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/BFM.zip -O ./checkpoints/BFM.zip
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/DNet.pt -O ./checkpoints/DNet.pt
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ENet.pth -O ./checkpoints/ENet.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/expression.mat -O ./checkpoints/expression.mat
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/face3d_pretrain_epoch_20.pth -O ./checkpoints/face3d_pretrain_epoch_20.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GFPGANv1.3.pth -O ./checkpoints/GFPGANv1.3.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GPEN-BFR-512.pth -O ./checkpoints/GPEN-BFR-512.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/LNet.pth -O ./checkpoints/LNet.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ParseNet-latest.pth -O ./checkpoints/ParseNet-latest.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/RetinaFace-R50.pth -O ./checkpoints/RetinaFace-R50.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/shape_predictor_68_face_landmarks.dat -O ./checkpoints/shape_predictor_68_face_landmarks.dat
!unzip -d ./checkpoints/BFM ./checkpoints/BFM.zip

Download pre-trained models...
mkdir: cannot create directory ‘./checkpoints’: File exists
--2024-12-12 21:24:11--  https://github.com/vinthony/video-retalking/releases/download/v0.0.1/30_net_gen.pth
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/OpenTalker/video-retalking/releases/download/v0.0.1/30_net_gen.pth [following]
--2024-12-12 21:24:11--  https://github.com/OpenTalker/video-retalking/releases/download/v0.0.1/30_net_gen.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/536411820/38c02c2b-bf57-4d4e-9711-6cefdc5c817c?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241212%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241212T212411Z&X-Amz-Expires=300&X-Amz-

**Inference**

`--face`: Input video.

`--audio`: Input audio. Both *.wav* and *.mp4* files are supported.

You can choose our provided data from `./examples` folder or upload from your local computer.






In [18]:
#@title
import glob, os, sys
import ipywidgets as widgets
from IPython.display import HTML
from base64 import b64encode

print("Choose the Video name to edit: (saved in folder 'examples/face')")
vid_list = glob.glob1('examples/face/', '*.mp4')
vid_list.sort()
default_vid_name = widgets.Dropdown(options=vid_list, value='1.mp4')
display(default_vid_name)

print("Choose the Audio name to edit: (saved in folder 'examples/audio')")
audio_list = glob.glob1('examples/audio/', '*')
audio_list.sort()
default_audio_name = widgets.Dropdown(options=audio_list, value='1.wav')
display(default_audio_name)


Choose the Video name to edit: (saved in folder 'examples/face')


Dropdown(options=('1.mp4', '2.mp4', '3.mp4', '4.mp4', '5.mp4'), value='1.mp4')

Choose the Audio name to edit: (saved in folder 'examples/audio')


Dropdown(options=('1.wav', '2.wav'), value='1.wav')

Visualize the input video and audio:

In [19]:
#@title
input_video_name = './examples/face/{}'.format(default_vid_name.value)
input_video_mp4 = open('{}'.format(input_video_name),'rb').read()
input_video_data_url = "data:video/x-m4v;base64," + b64encode(input_video_mp4).decode()
print('Display input video: {}'.format(input_video_name), file=sys.stderr)
display(HTML("""
  <video width=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % input_video_data_url))

input_audio_name = './examples/audio/{}'.format(default_audio_name.value)
input_audio_mp4 = open('{}'.format(input_audio_name),'rb').read()
input_audio_data_url = "data:audio/wav;base64," + b64encode(input_audio_mp4).decode()
print('Display input audio: {}'.format(input_audio_name), file=sys.stderr)
display(HTML("""
  <audio width=400 controls>
        <source src="%s" type="audio/wav">
  </audio>
  """ % input_audio_data_url))


Display input video: ./examples/face/1.mp4


Display input audio: ./examples/audio/1.wav


In [20]:
input_video_path = 'examples/face/{}'.format(default_vid_name.value)
input_audio_path = 'examples/audio/{}'.format(default_audio_name.value)

!python3 inference.py \
  --face {input_video_path} \
  --audio {input_audio_path} \
  --outfile results/output.mp4

[Info] Using cuda for inference.
[Step 0] Number of frames available for inference: 135
[Step 1] Using saved landmarks.
[Step 2] 3DMM Extraction In Video:: 100% 135/135 [00:01<00:00, 74.75it/s] 
using expression center
Load checkpoint from: checkpoints/DNet.pt
Load checkpoint from: checkpoints/LNet.pth
Load checkpoint from: checkpoints/ENet.pth
[Step 3] Using saved stabilized video.
[Step 4] Load audio; Length of mel chunks: 93
[Step 5] Reference Enhancement: 100% 93/93 [00:29<00:00,  3.16it/s]
[Step 6] Lip Synthesis::   0% 0/6 [00:00<?, ?it/s]
landmark Det::   0% 0/93 [00:00<?, ?it/s]
landmark Det::   1% 1/93 [00:06<10:04,  6.58s/it]
landmark Det::   2% 2/93 [00:13<10:37,  7.00s/it]
landmark Det::   4% 4/93 [00:13<03:55,  2.64s/it]
landmark Det::   8% 7/93 [00:14<01:39,  1.15s/it]
landmark Det::  11% 10/93 [00:14<00:53,  1.54it/s]
landmark Det::  15% 14/93 [00:14<00:28,  2.75it/s]
landmark Det::  19% 18/93 [00:14<00:17,  4.35it/s]
landmark Det::  24% 22/93 [00:14<00:11,  6.37it/s]
lan

Visualize the output video:

In [21]:
#@title
# visualize code from makeittalk
from IPython.display import HTML
from base64 import b64encode
import os, sys, glob, cv2, subprocess, platform

def read_video(vid_name):
  video_stream = cv2.VideoCapture(vid_name)
  fps = video_stream.get(cv2.CAP_PROP_FPS)
  full_frames = []
  while True:
    still_reading, frame = video_stream.read()
    if not still_reading:
        video_stream.release()
        break
    full_frames.append(frame)
  return full_frames, fps

input_video_frames, fps = read_video(input_video_path)
output_video_frames, _ = read_video('./results/output.mp4')

frame_h, frame_w = input_video_frames[0].shape[:-1]
out_concat = cv2.VideoWriter('./temp/temp/result_concat.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_w*2, frame_h))
for i in range(len(output_video_frames)):
  frame_input = input_video_frames[i % len(input_video_frames)]
  frame_output = output_video_frames[i]
  out_concat.write(cv2.hconcat([frame_input, frame_output]))
out_concat.release()

command = 'ffmpeg -loglevel error -y -i {} -i {} -strict -2 -q:v 1 {}'.format(input_audio_path, './temp/temp/result_concat.mp4', './results/output_concat_input.mp4')
subprocess.call(command, shell=platform.system() != 'Windows')


output_video_name = './results/output.mp4'
output_video_mp4 = open('{}'.format(output_video_name),'rb').read()
output_video_data_url = "data:video/mp4;base64," + b64encode(output_video_mp4).decode()
print('Display lip-syncing video: {}'.format(output_video_name), file=sys.stderr)
display(HTML("""
  <video height=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % output_video_data_url))

output_concat_video_name = './results/output_concat_input.mp4'
output_concat_video_mp4 = open('{}'.format(output_concat_video_name),'rb').read()
output_concat_video_data_url = "data:video/mp4;base64," + b64encode(output_concat_video_mp4).decode()
print('Display input video and lip-syncing video: {}'.format(output_concat_video_name), file=sys.stderr)
display(HTML("""
  <video height=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % output_concat_video_data_url))


Display lip-syncing video: ./results/output.mp4


Display input video and lip-syncing video: ./results/output_concat_input.mp4


In [14]:
pip install torch==1.13.0 torchvision==0.14.0


  Using cached torch-1.13.0-cp310-cp310-manylinux1_x86_64.whl.metadata (23 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation

In [15]:
pip install torch==1.12.0 torchvision==0.13.0

  Using cached torch-1.12.0-cp310-cp310-manylinux1_x86_64.whl.metadata (22 kB)
  Using cached torchvision-0.13.0-cp310-cp310-manylinux1_x86_64.whl.metadata (10 kB)
Using cached torch-1.12.0-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
Using cached torchvision-0.13.0-cp310-cp310-manylinux1_x86_64.whl (19.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0
    Uninstalling torchvision-0.14.0:
      Successfully uninstalled torchvision-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.12.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.12.0 which is incompatible.


# New Section

In [11]:
pip install --upgrade torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━